# Deploy Model

# Setup

In [1]:
# %pip install -r requirements.txt

In [2]:
# Environment Variables
import os
from dotenv import load_dotenv

# Load env
load_dotenv()

True

# Reference

https://www.hiberus.com/crecemos-contigo/ask-your-web-pages-otro-enfoque-rag-utilizando-modelos-de-codigo-abierto/

https://colab.research.google.com/drive/1rt318Ew-5dDw21YZx2zK2vnxbsuDAchH?usp=sharing#scrollTo=YFw8HWIyTCnJ

https://www.reddit.com/r/LocalLLaMA/comments/16j624z/some_questions_of_implementing_llm_to_generate_qa/

https://www.anyscale.com/blog/a-comprehensive-guide-for-building-rag-based-llm-applications-part-1

https://towardsdatascience.com/rag-how-to-talk-to-your-data-eaf5469b83b0

https://github.com/edumunozsala/question-answering-pinecone-sts

# Directory

In [3]:
# Set directory to file location
from pathlib import Path
import sys
notebook_location = Path(os.path.abspath(''))
os.chdir(notebook_location)
# Get the current working directory
current_directory = os.getcwd()
current_directory

'/notebooks/LawGPT'

# Libraries

In [4]:
# General
from IPython.display import Markdown, display
import gradio as gr
import pinecone
import yaml
import time
import json

import gc
import os

# HuggingFace
from huggingface_hub import notebook_login

# Transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import transformers

# Langchain
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import SimpleSequentialChain, RetrievalQA, LLMChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
from langchain import HuggingFacePipeline
from langchain import PromptTemplate

# Torch
from torch import cuda, bfloat16, float16
import torch

# Other
from tqdm.notebook import tqdm

# Local
from functions import *

# Warnings
import warnings
warnings.filterwarnings("ignore")

# Platform login

Use credentials from HuggingFace

In [5]:
# HF Key
hf_key = os.environ.get('HF_KEY')

In [6]:
# Jupyter / Colab
# notebook_login()

# VS Code
# Run huggingface-cli login in console

In [7]:
# Setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# CUDA information
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Quadro P5000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [8]:
# Clean memory
torch.cuda.empty_cache()
gc.collect()

248

# Pinecone

In [9]:
# Init pinecone
pinecone.init(
    api_key = os.environ.get('PINECONE_API_KEY'),
    environment = os.environ.get('PINECONE_ENVIRONMENT')
)

# Connect
index_name = 'lawgpt-unstructured-db'
index = pinecone.Index(index_name)

# Index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04627,
 'namespaces': {'': {'vector_count': 4627}},
 'total_vector_count': 4627}

# Parameters

In [10]:
# Load parameters from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Embedding model

In [11]:
# Model ID
embed_model_id = config["embedding_model"]

# Embed model
embed_model = HuggingFaceEmbeddings(
    model_name = embed_model_id,
    model_kwargs = {'device': device},
    encode_kwargs = {'device': device, 'batch_size': 32}
) 

# Load LLM model

In [12]:
# Select model
model_id = config["model"]

# BNB Config
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = bfloat16
)

# Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Set model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code = True,
    quantization_config = bnb_config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
# CUDA information
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Quadro P5000
Memory Usage:
Allocated: 4.3 GB
Cached:    4.5 GB


# Command

In [14]:
# Get pre-prompt
pre_prompt = config["pre_prompt"]

# Create prompt context
prompt_context = config["prompt_context"]

In [15]:
# General template
prompt_template = pre_prompt + prompt_context + "A continuación se proporciona el contexto: {context}" + " " + "pregunta: {query}"

In [16]:
# Mistral template
mistral_template = "[INST]" + pre_prompt + prompt_context +  "A continuación se proporciona el contexto: {context}" + " " + "pregunta: {query}" + "[/INST]"

In [17]:
# Final template
if "mistral" in model_id.lower():
    final_template = mistral_template
else: 
    final_template = general_template

# Prompt Template
prompt = PromptTemplate(
    template = final_template, 
    input_variables = ["context", "query"]
)

# LLM Pipeline

In [18]:
# Define pipeline with parameters from JSON file
generate_text = transformers.pipeline(
    model = model,
    tokenizer = tokenizer,
    task = 'text-generation',
    temperature = config["temperature"],
    repetition_penalty = config["repetition_penalty"],
    return_full_text = config["return_full_text"],
    max_new_tokens = config["max_new_tokens"],
    pad_token_id = tokenizer.eos_token_id
)

# HF pipeline
llm = HuggingFacePipeline(pipeline = generate_text)

# Create llm chain 
llm_chain = LLMChain(llm = llm, prompt = prompt)

# Vector store

In [19]:
# Field in metadata with text
text_field = 'text'

# Initiate langchain vectorstore
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

# RAG Chain

In [20]:
# RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm = llm, 
    retriever = vectorstore.as_retriever(search_kwargs= {"k": config['top_k_docs']}),
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt},
    chain_type = 'stuff'
)

# Test models

In [21]:
# Simple context
context = "Eres una API con conocimientos legales. Debes responder a preguntas en Español. Si no conoces la respuesta, admítelo."

# Query
query = '¿Qué diferencias existen entre los dos tipos delictivos que el código penal regula en el artículo 245?'

# Generic model

In [22]:
# Default LLM
generic_model = llm_chain.invoke({"context": context, "query": query})

In [23]:
# Output
generic_result = generic_model['text'].strip()

# Markdown
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{generic_result}</p>"))

<b>¿Qué diferencias existen entre los dos tipos delictivos que el código penal regula en el artículo 245?</b>

<p>Context: El Código Penal Español establece dos tipos de delitos en el artículo 245: "delito contra la seguridad exterior de España" y "delito contra la seguridad interior de España".

Respuesta: Los delitos contra la seguridad exterior y contra la seguridad interior son dos tipos distintos de delitos regulados en el artículo 245 del Código Penal Español.

Los delitos contra la seguridad exterior (artículo 245.1) se refieren a las acciones que atentan contra la integridad territorial de España o contra sus instituciones diplomáticas o consulares. Estos delitos pueden incluir actos de agresión militar, piratería, espionaje o sabotaje.

En contraste, los delitos contra la seguridad interior (artículo 245.2) se refieren a las acciones que amenazan la seguridad interna de España o de sus instituciones. Estos delitos pueden incluir actos terroristas, actos de sabotaje, actos de guerra civil o actos de lesa majestad.

Las principales diferencias entre ambos tipos de delitos radican en su objetivo y en el ámbito geográfico en que se cometen. Delitos contra la seguridad exterior atentan contra la integridad territorial de España o contra sus instituciones diplomáticas o consulares, lo que implica una dimensión internacional. En cambio, los delitos contra la seguridad interior amenazan la seguridad interna de España o de sus instituciones, lo que implica una dimensión nacional.

Fuente: Código Penal Español. Ley 31/1978, de 1 de octubre. BOE núm. 269, de 2 de noviembre de 1978.</p>

# RAG chain

In [24]:
# RAG chain
rag_model = rag_chain.invoke({"query": query})

ValueError: Missing some input keys: {'query'}

In [ ]:
# Output
rag_result = rag_model['result'].strip()

# Markdown
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{rag_result}</p>"))